## COD API, Matches --> explore, format, clean & reshape

Activision Call of Duty API use case for **Matches** endpoint, using a slightly amended version of callofduty.py client and custom cleaning functions (from wzkd app)

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

#### Login using SSO

In [ ]:
# We're storing our SSO token in an .env file stored locally to separate our conf from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### Slightly modify client methods to call the API matches endpoint

In [ ]:
# This time we're adding additional methodsin the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use

# 1. Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# 2. following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['matches'] # API res was filtered out here


async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# 3. add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

#### Get Matches data

In [ ]:
# In this notebook, we will only use the most interesting api's endpoint /...details, with the method Client.GetMatchesDetailed
# raw_matches_datailed has indeed two keys : summary and matches (the one that is filtered out in the Client)
matches = await client.GetMatchesDetailed("battle", "amadevs#1689", Title.ModernWarfare, Mode.Warzone, limit=20) #AMADEVS#1689

##### Option: save previous result so we're not getting annoyed by API rate limits or inconsistencies -,-

In [ ]:
with open("matches.pkl", 'wb') as f:
    pickle.dump(matches, f)

## Match result : structure

In [ ]:
# load previously saved data
with open('matches.pkl', 'rb') as f:
    matches = pickle.load(f)

### Overview : dict --> df

In [ ]:
df_matches = pd.DataFrame(matches)
display(df_matches.head(2))
keys = list(df_matches.keys())
keys.sort()
pprint(keys)

### API structure for one match (a 'row')

##### Reminder : matches endpoint returns n (max 20) matches & their associated stats, as 'rows', for a single --queried player

In [ ]:
pprint(matches[10], depth=2)

### Focus : what's in 'playerStats' ?

In [ ]:
player_stats = df_matches['playerStats'].apply(pd.Series)
display(player_stats.head(5))
pprint(player_stats.keys())

In [ ]:
displayed_cols = ['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']
# Just in case a specific game mode does not contain a value we wanted to display
displayed_cols = [col for col in displayed_cols if col in list(player_stats.keys())]
player_stats[displayed_cols].head(5)

### Focus : 'player', a nested entry

In [ ]:
player = df_matches['player'].apply(pd.Series)
display(player.head(5))
pprint(player.keys())

#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [ ]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
match_index = 2
pprint(player['loadout'][match_index][0], depth=2)

#### Inside 'player' entry, 'brMissionStats' is a (list of) dict

In [ ]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
match_index = 4
pprint(player['brMissionStats'][match_index], depth=3)

## Format & clean API **matches** result using customized tools (wzkd app)

In [2]:
import json
import toml
# functions defined in wzkd app directory '/wzkd/wzkd'
sys.path.insert(0, os.path.abspath('../wzkd'))
from utils import load_labels, load_conf
from api_format import res_to_df, format_df

In [3]:
# conf and labels files stored here as well.
# labels is needed for parsing games modes/weapons, conf stores values such as n of loadouts to extract or columns names
file_labels = "wz_labels.json"
filepath_labels = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/wzkd/" + file_labels
LABELS = load_labels(filepath_labels)
pprint(LABELS, depth=2)

file_conf = "conf.toml"
filepath_conf = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/wzkd/" + file_conf
CONF = load_conf(filepath_conf)
pprint(CONF, depth=2)

{'modes': {'battle_royale': {...}, 'multiplayer': {...}},
 'weapons': {'cat_names': {...},
             'categories': [...],
             'names': {...},
             'prefixes': [...]}}
{'API_OUTPUT_FORMAT': {'float_cols': [...],
                       'int_cols': [...],
                       'mission_types': [...],
                       'n_loadouts': 3,
                       'ts_cols': [...]},
 'APP_BEHAVIOR': {'br_only': True, 'mode': 'offline'},
 'APP_DISPLAY': {'keep_cols': {...}, 'labels': {...}}}


In [19]:
with open('matches.pkl', 'rb') as f:
    res = pickle.load(f)
tmp = pd.DataFrame(res)
display(tmp.head(2))
tmp.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1650580154,1650581565,mp_wz_island,br_brduos,14049601834280347023,1411000,None,1,wz,152,"{'kills': 1.0, 'medalXp': 20.0, 'matchXp': 451...","{'team': 'team_twenty_four', 'rank': 54.0, 'aw...",76,None,False,False
1,1650579855,1650581475,mp_wz_island,br_brduos,9318486774444753860,1620000,None,1,wz,151,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 542....","{'team': 'team_twenty_six', 'rank': 54.0, 'awa...",76,None,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   utcStartSeconds  20 non-null     int64 
 1   utcEndSeconds    20 non-null     int64 
 2   map              20 non-null     object
 3   mode             20 non-null     object
 4   matchID          20 non-null     object
 5   duration         20 non-null     int64 
 6   playlistName     0 non-null      object
 7   version          20 non-null     int64 
 8   gameType         20 non-null     object
 9   playerCount      20 non-null     int64 
 10  playerStats      20 non-null     object
 11  player           20 non-null     object
 12  teamCount        20 non-null     int64 
 13  rankedTeams      0 non-null      object
 14  draw             20 non-null     bool  
 15  privateMatch     20 non-null     bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 2.4+ KB


In [20]:
# flatten-expand into a DataFrame the result from COD API, for matches history
df_matches = res_to_df(res, CONF)
display(df_matches.head(5))
df_matches.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_1,loadout_2,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,assassination,domination,timedrun,scavenger
0,1650580154,1650581565,mp_wz_island,br_brduos,14049601834280347023,1411000,1,wz,152,76,...,"{'primaryWeapon': {'name': 's4_sm_mpapa40', 'l...","{'primaryWeapon': {'name': 's4_pi_mike1911', '...",NaN,0,0.0,0.0,NaN,NaN,NaN,NaN
1,1650579855,1650581475,mp_wz_island,br_brduos,9318486774444753860,1620000,1,wz,151,76,...,"{'primaryWeapon': {'name': 's4_sm_mpapa40', 'l...","{'primaryWeapon': {'name': 's4_pi_mike1911', '...",NaN,0,0.0,0.0,NaN,NaN,NaN,NaN
2,1650579021,1650580593,mp_wz_island,br_brduos,7917282986023863146,1572000,1,wz,150,75,...,"{'primaryWeapon': {'name': 's4_sm_mpapa40', 'l...","{'primaryWeapon': {'name': 's4_pi_mike1911', '...",NaN,1,1000.0,1000.0,"{'weaponXp': 1000.0, 'xp': 1000.0, 'count': 1.0}",NaN,NaN,NaN
3,1650577338,1650578903,mp_wz_island,br_brduos,11927080532369025479,1565000,1,wz,149,76,...,"{'primaryWeapon': {'name': 's4_sm_mpapa40', 'l...",NaN,NaN,1,575.0,575.0,NaN,"{'weaponXp': 575.0, 'xp': 575.0, 'count': 1.0}",NaN,NaN
4,1650576573,1650578143,mp_wz_island,br_brduos,17475147476072357324,1570000,1,wz,150,77,...,"{'primaryWeapon': {'name': 's4_sm_mpapa40', 'l...",NaN,NaN,1,500.0,1000.0,NaN,"{'weaponXp': 1000.0, 'xp': 500.0, 'count': 1.0}",NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   utcStartSeconds             20 non-null     int64  
 1   utcEndSeconds               20 non-null     int64  
 2   map                         20 non-null     object 
 3   mode                        20 non-null     object 
 4   matchID                     20 non-null     object 
 5   duration                    20 non-null     int64  
 6   version                     20 non-null     int64  
 7   gameType                    20 non-null     object 
 8   playerCount                 20 non-null     int64  
 9   teamCount                   20 non-null     int64  
 10  draw                        20 non-null     bool   
 11  privateMatch                20 non-null     bool   
 12  kills                       20 non-null     float64
 13  medalXp                     20 non-nu

In [21]:
# make the stats human-readable and parse some values (weapons, games modes)
df_formatted = format_df(df_matches,CONF, LABELS)
display(df_formatted.head(5))
df_formatted.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_1,loadout_2,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,assassination,domination,timedrun,scavenger
0,2022-04-22 00:29:14,2022-04-22 00:52:45,mp_wz_island,Duos,14049601834280347023,23,1,wz,152,76,...,MP40 K31,M19 fists,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,<NA>
1,2022-04-22 00:24:15,2022-04-22 00:51:15,mp_wz_island,Duos,9318486774444753860,27,1,wz,151,76,...,MP40 K31,M19 fists,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,<NA>
2,2022-04-22 00:10:21,2022-04-22 00:36:33,mp_wz_island,Duos,7917282986023863146,26,1,wz,150,75,...,MP40 K31,M19 fists,NaN,1,1000.0,1000.0,1,<NA>,<NA>,<NA>
3,2022-04-21 23:42:18,2022-04-22 00:08:23,mp_wz_island,Duos,11927080532369025479,26,1,wz,149,76,...,MP40 M82,NaN,NaN,1,575.0,575.0,<NA>,1,<NA>,<NA>
4,2022-04-21 23:29:33,2022-04-21 23:55:43,mp_wz_island,Duos,17475147476072357324,26,1,wz,150,77,...,MP40 M82,NaN,NaN,1,500.0,1000.0,<NA>,1,<NA>,<NA>


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   utcStartSeconds             20 non-null     datetime64[ns]
 1   utcEndSeconds               20 non-null     datetime64[ns]
 2   map                         20 non-null     object        
 3   mode                        20 non-null     object        
 4   matchID                     20 non-null     object        
 5   duration                    20 non-null     object        
 6   version                     20 non-null     int64         
 7   gameType                    20 non-null     object        
 8   playerCount                 20 non-null     int64         
 9   teamCount                   20 non-null     Int64         
 10  draw                        20 non-null     bool          
 11  privateMatch                20 non-null     bool          
 

#### tests